In [1]:
pip install torch>=1.13.0+cu116 torchvision>=0.13.0+cu116 torchaudio>=0.13.0 --extra-index-url https://download.pytorch.org/whl/cu116

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_recall_fscore_support,confusion_matrix
import numpy as np
import os
import matplotlib.pyplot as plt
import random
import seaborn as sns

In [5]:
model = torch.load("final_model_inceptionv3(Dataset1).pth")

C:\Users\singh\AppData\Local\Temp\ipykernel_16240\2094001544.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("final_model_inceptionv3(Dataset1).pth")


In [7]:
model.eval() 

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [9]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [13]:
dataset_path = r'D:\Image_dataset\animal_computer_vision\Dataset'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
class_names = dataset.classes
print(class_names)

['Buffalo', 'Elephant', 'Rhino', 'Zebra']


In [15]:

train_size = int(0.7 * len(dataset))  
val_size = int(0.15 * len(dataset))  
test_size = len(dataset) - train_size - val_size  

In [17]:

train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])

In [19]:

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [21]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
from itertools import product

learning_rates = [0.001, 0.005, 0.01, 0.05, 0.1]
batch_sizes = [16, 32, 64]
combinations = list(product(learning_rates, batch_sizes))

results = {}

patience = 3  
for lr, batch_size in combinations:
    print(f"Training with learning rate: {lr} and batch size: {batch_size}")
    
    train_size = int(0.7 * len(dataset))
    val_size = int(0.15 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])
    
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    
    model = models.inception_v3(pretrained=False)  
    model.aux_logits = False
    model.fc = nn.Linear(model.fc.in_features, 4)
    model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    num_epochs = 15
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):

        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {running_loss / len(train_loader)}")
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_loss}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0  
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch + 1}")
                break
    
    val_accuracy = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_accuracy += (preds == labels).sum().item()
            total += labels.size(0)
    
    accuracy = val_accuracy / total
    results[(lr, batch_size)] = accuracy
    print(f"Validation Accuracy: {accuracy}")
    
    test_accuracy = 0
    total_test = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            test_accuracy += (preds == labels).sum().item()
            total_test += labels.size(0)
    
    test_acc = test_accuracy / total_test
    print(f"Test Accuracy for LR: {lr}, Batch Size: {batch_size} -> {test_acc}")


best_params = max(results, key=results.get)
print(f"Best Learning Rate: {best_params[0]}, Best Batch Size: {best_params[1]} with Accuracy: {results[best_params]}")


Training with learning rate: 0.001 and batch size: 16


C:\Users\singh\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\singh\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\singh\AppData\Roaming\Python\Python312\site-packages\torchvision\models\inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


Epoch 1/15, Training Loss: 0.9437575975486211
Epoch 1/15, Validation Loss: 0.8362907767295837
Epoch 2/15, Training Loss: 0.7383215067216328
Epoch 2/15, Validation Loss: 0.8619713536218593
Epoch 3/15, Training Loss: 0.6389461361510413
Epoch 3/15, Validation Loss: 0.6329651690627399
Epoch 4/15, Training Loss: 0.5902345250334059
Epoch 4/15, Validation Loss: 0.7209975703766471
Epoch 5/15, Training Loss: 0.5116347193717956
Epoch 5/15, Validation Loss: 0.5403076088987291
Epoch 6/15, Training Loss: 0.4475201259340559
Epoch 6/15, Validation Loss: 0.42327855870519815
Epoch 7/15, Training Loss: 0.42536894070250647
Epoch 7/15, Validation Loss: 0.45842087288436134
Epoch 8/15, Training Loss: 0.4149013264264379
Epoch 8/15, Validation Loss: 0.40627461936520903
Epoch 9/15, Training Loss: 0.3664476606249809
Epoch 9/15, Validation Loss: 0.43537265915227563
Epoch 10/15, Training Loss: 0.3069357124609607
Epoch 10/15, Validation Loss: 0.2936454331012149
Epoch 11/15, Training Loss: 0.33930886806121896
Epoch